## NanoDet 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# --- Cell 2: Setup NanoDet ---

# Clone the repository
!git clone https://github.com/RangiLyu/nanodet.git

# Install requirements
!pip install -q /kaggle/working/nanodet/

# Change the current working directory to 'nanodet'
# This is important so the script can find its files
%cd /kaggle/working/nanodet

In [ ]:
# --- Cell 3: Create Dataset Config ---
import os

# Content for the apron_dataset.yml file
# It uses the paths from your Kaggle environment
dataset_yaml_content = """
class_names: &class_names [
    "Safety Vehicle", "Business Aircraft", "Cargo Aircraft", "Common Aircraft", 
    "Container Trolley", "Conveyer Vehicle", "Helicopter", "Loading Ramp", 
    "Loading Vehicle", "Other Objects", "Other Vehicles", "Passenger Bus", 
    "Passenger Stairway", "Person", "Standard Car", "Tank Truck", 
    "Taxiing Vehicle", "Traffic Barrier", "Traffic Cone", "Traffic Cone Light", 
    "Traffic Sign", "Transport Container", "Transport Vehicle"
]

train:
  name: CocoDataset
  img_path: /kaggle/input/airportvehicledataset/images
  ann_path: /kaggle/working/coco_dataset/train.json
  input_size: [416, 416]
  keep_ratio: True
  pipeline:
    perspective: 0.0
    scale: [0.5, 1.5]
    stretch: [[1, 1], [1, 1]]
    rotation: 0
    shear: 0
    translate: 0.2
    flip: 0.5
    brightness: 0.2
    contrast: [0.8, 1.2]
    saturation: [0.8, 1.2]
    normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]

val:
  name: CocoDataset
  img_path: /kaggle/input/airportvehicledataset/images
  ann_path: /kaggle/working/coco_dataset/val.json
  input_size: [416, 416]
  keep_ratio: True
  pipeline:
    normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]
"""

# Write the content to the correct location
config_path = 'config/dataset/apron_dataset.yml'
os.makedirs(os.path.dirname(config_path), exist_ok=True)
with open(config_path, 'w') as f:
    f.write(dataset_yaml_content)

print(f"✅ Created dataset config at: {config_path}")

In [ ]:
# --- Cell 4: Create Model Config ---

# Full content for the nanodet-m-apron.yml file
# This is a modified version of the default nanodet-m config
model_yaml_content = """
# The directory to save logs and models
save_dir: /kaggle/working/workspace/nanodet_m_apron

# Inherit from our custom dataset config
_base_: ../dataset/apron_dataset.yml

model:
  arch:
    name: NanoDetPlus
    backbone:
      name: ShuffleNetV2
      model_size: 1.0x
      out_stages: [2,3,4]
      activation: LeakyReLU
    fpn:
      name: GhostPAN
      in_channels: [116, 232, 464]
      out_channels: 96
      kernel_size: 5
      num_extra_level: 1
      use_depthwise: True
      activation: LeakyReLU
    head:
      name: NanoDetPlusHead
      num_classes: 23  # <--- CRITICAL: Changed from 80 to 23
      input_channel: 96
      feat_channels: 96
      stacked_convs: 2
      kernel_size: 5
      strides: [8, 16, 32, 64]
      conv_type: dws
      activation: LeakyReLU
      reg_max: 7
    # Auxiliary head for NNDet-Plus
    aux_head:
      name: SimpleConvHead
      num_classes: 23 # <--- CRITICAL: Changed from 80 to 23
      input_channel: 192 # 96+96
      feat_channels: 96
      stacked_convs: 4
      strides: [8, 16, 32, 64]
      activation: LeakyReLU
      reg_max: 7

class_names: *class_names
data:
  train:
    input_size: [416,416]
  val:
    input_size: [416,416]

device:
  batch_size_per_gpu: 16 # <-- Lower this (e.g., 8, 4) if you get a CUDA memory error
  gpu_ids: [0]

schedule:
  resume: null
  load_model: null
  optimizer:
    name: AdamW
    lr: 0.001
    weight_decay: 0.05
  warmup:
    name: linear
    steps: 500
    ratio: 0.0001
  total_epochs: 120 # <-- You can change the number of epochs to train
  lr_schedule:
    name: CosineAnnealingLR
    T_max: 120
    eta_min: 0.00005
  val_intervals: 10

evaluator:
  name: CocoDetectionEvaluator
  save_key: mAP

log:
  interval: 50
"""

# Write the content to the model config file
with open('config/nanodet-m-apron.yml', 'w') as f:
    f.write(model_yaml_content)

print("✅ Created model config at: config/nanodet-m-apron.yml")

In [ ]:
# --- Final Training Cell (using PYTHONPATH) ---
import os

print("File structure is correct. Applying the PYTHONPATH fix.")

# The project root directory
project_root = '/kaggle/working/nanodet'

# Change to the project directory (still good practice for relative paths in configs)
os.chdir(project_root)
print(f"Current Directory: {os.getcwd()}")

# --- The Command that Fixes the Issue ---
# We set the PYTHONPATH environment variable for this specific command.
# This tells the Python interpreter to add '/kaggle/working' to its
# search path, allowing it to find the 'nanodet' module.
!PYTHONPATH=/kaggle/working python tools/train.py -c config/nanodet-m-apron.yml

In [ ]:
# --- Cell to modify train.py in-place ---
import os

# The path to the script we need to modify
script_path = '/kaggle/working/nanodet/tools/train.py'

# The lines of code to insert at the top of the script.
# This forces the script to see the '/kaggle/working' directory.
code_to_insert = "import sys\nsys.path.insert(0, '/kaggle/working')\n\n"

print(f"Attempting to modify {script_path}...")

try:
    # Read the original content of the script
    with open(script_path, 'r') as f:
        original_content = f.read()

    # Write the new content (our code + original code) back to the file
    with open(script_path, 'w') as f:
        f.write(code_to_insert + original_content)

    print("✅ Script modified successfully.")
    
except Exception as e:
    print(f"❌ An error occurred: {e}")

In [ ]:
# --- Final Run Cell ---

# Change to the project directory
%cd /kaggle/working/nanodet

# Run the training script. It is now hard-coded to find the library.
!python tools/train.py -c config/nanodet-m-apron.yml

In [16]:
%%bash
set -e

# --- STEP 1: CLEANUP ---
echo "--- Step 1: Cleaning up environment ---"
pip uninstall -y nanodet || echo "nanodet was not installed, which is good."
rm -rf /kaggle/nanodet
echo "Cleanup complete."

# --- STEP 2: CLONE & ISOLATED INSTALL ---
echo -e "\n--- Step 2: Cloning and Creating an Isolated Installation ---"
git clone https://github.com/RangiLyu/nanodet.git /kaggle/nanodet
cd /kaggle/nanodet

# --- THE ISOLATION FIX ---
# We use --ignore-installed to force pip to install fresh packages and
# ignore the conflicting ones pre-installed on Kaggle.

echo "Installing dependencies into an isolated user environment..."

# We don't need to sanitize the requirements file anymore, we just need to install them.
# The --ignore-installed flag is the key.
pip install --ignore-installed -r requirements.txt

# Manually install a compatible mmcv-full, also ignoring system packages
pip install --ignore-installed "mmcv-full<2.0.0,>=1.7.0" -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html

# Now, build and install nanodet itself.
# --no-deps tells it not to re-verify dependencies we just installed.
echo "Building and installing nanodet..."
pip install --ignore-installed --no-deps .
echo "✅ NanoDet package and all dependencies installed successfully in isolation."

# --- STEP 3: CREATE CONFIG FILES ---
echo -e "\n--- Step 3: Creating configuration files ---"
mkdir -p config/dataset

cat <<'EOF' > config/dataset/apron_dataset.yml
class_names: &class_names [
    "Safety Vehicle", "Business Aircraft", "Cargo Aircraft", "Common Aircraft", 
    "Container Trolley", "Conveyer Vehicle", "Helicopter", "Loading Ramp", 
    "Loading Vehicle", "Other Objects", "Other Vehicles", "Passenger Bus", 
    "Passenger Stairway", "Person", "Standard Car", "Tank Truck", 
    "Taxiing Vehicle", "Traffic Barrier", "Traffic Cone", "Traffic Cone Light", 
    "Traffic Sign", "Transport Container", "Transport Vehicle"
]
train:
  name: CocoDataset
  img_path: /kaggle/input/airportvehicledataset/images
  ann_path: /kaggle/coco_dataset/train.json
  input_size: [416, 416]
  keep_ratio: True
  pipeline:
    perspective: 0.0
    scale: [0.5, 1.5]
    stretch: [[1, 1], [1, 1]]
    rotation: 0
    shear: 0
    translate: 0.2
    flip: 0.5
    brightness: 0.2
    contrast: [0.8, 1.2]
    saturation: [0.8, 1.2]
    normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]
val:
  name: CocoDataset
  img_path: /kaggle/input/airportvehicledataset/images
  ann_path: /kaggle/coco_dataset/val.json
  input_size: [416, 416]
  keep_ratio: True
  pipeline:
    normalize: [[103.53, 116.28, 123.675], [57.375, 57.12, 58.395]]
EOF
echo "Created apron_dataset.yml"

cat <<'EOF' > config/nanodet-m-apron.yml
save_dir: /kaggle/workspace/nanodet_m_apron
_base_: ../dataset/apron_dataset.yml
model:
  arch:
    name: NanoDetPlus
    backbone:
      name: ShuffleNetV2
      model_size: 1.0x
      out_stages: [2,3,4]
      activation: LeakyReLU
    fpn:
      name: GhostPAN
      in_channels: [116, 232, 464]
      out_channels: 96
      kernel_size: 5
      num_extra_level: 1
      use_depthwise: True
      activation: LeakyReLU
    head:
      name: NanoDetPlusHead
      num_classes: 23
      input_channel: 96
      feat_channels: 96
      stacked_convs: 2
      kernel_size: 5
      strides: [8, 16, 32, 64]
      conv_type: dws
      activation: LeakyReLU
      reg_max: 7
    aux_head:
      name: SimpleConvHead
      num_classes: 23
      input_channel: 192
      feat_channels: 96
      stacked_convs: 4
      strides: [8, 16, 32, 64]
      activation: LeakyReLU
      reg_max: 7
class_names: *class_names
data:
  train:
    input_size: [416,416]
  val:
    input_size: [416,416]
device:
  batch_size_per_gpu: 8
  gpu_ids: [0]
schedule:
  resume: null
  load_model: null
  optimizer:
    name: AdamW
    lr: 0.001
    weight_decay: 0.05
  warmup:
    name: linear
    steps: 500
    ratio: 0.0001
  total_epochs: 120
  lr_schedule:
    name: CosineAnnealingLR
    T_max: 120
    eta_min: 0.00005
  val_intervals: 10
evaluator:
  name: CocoDetectionEvaluator
  save_key: mAP
log:
  interval: 50
EOF
echo "Created nanodet-m-apron.yml"

# --- STEP 4: TRAIN ---
echo -e "\n--- Step 4: Starting the training ---"
python tools/train.py -c config/nanodet-m-apron.yml

--- Step 1: Cleaning up environment ---
Cleanup complete.

--- Step 2: Cloning and Creating an Isolated Installation ---
Installing dependencies into an isolated user environment...
  Using cached cython-3.1.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.9 kB)
  Using cached imagesize-1.4.1-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached matplotlib-3.10.5-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached numpy-2.3.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached onnx-1.18.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
  Using cached onnx_simplifier-0.4.36-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached pyaml-25.7.0-py3-

Cloning into '/kaggle/nanodet'...
ERROR: Cannot install -r requirements.txt (line 11), -r requirements.txt (line 16) and torch<2.0 and >=1.10 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


CalledProcessError: Command 'b'set -e\n\n# --- STEP 1: CLEANUP ---\necho "--- Step 1: Cleaning up environment ---"\npip uninstall -y nanodet || echo "nanodet was not installed, which is good."\nrm -rf /kaggle/nanodet\necho "Cleanup complete."\n\n# --- STEP 2: CLONE & ISOLATED INSTALL ---\necho -e "\\n--- Step 2: Cloning and Creating an Isolated Installation ---"\ngit clone https://github.com/RangiLyu/nanodet.git /kaggle/nanodet\ncd /kaggle/nanodet\n\n# --- THE ISOLATION FIX ---\n# We use --ignore-installed to force pip to install fresh packages and\n# ignore the conflicting ones pre-installed on Kaggle.\n\necho "Installing dependencies into an isolated user environment..."\n\n# We don\'t need to sanitize the requirements file anymore, we just need to install them.\n# The --ignore-installed flag is the key.\npip install --ignore-installed -r requirements.txt\n\n# Manually install a compatible mmcv-full, also ignoring system packages\npip install --ignore-installed "mmcv-full<2.0.0,>=1.7